<h2>Menu Items Extraction and Menu Categorisation</h2>

In [4]:
import os
import base64
import re
from anthropic import Anthropic
from collections import defaultdict
import logging # Import logging
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def encode_image(image_path):
    """
    Encode an image file to base64.

    :param image_path: Path to the image file
    :return: Base64 encoded string of the image
    """
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        logging.error(f"Image file not found: {image_path}")
        return None
    except Exception as e:
        logging.error(f"Error encoding image {image_path}: {e}")
        return None

def extract_text_from_image(api_key, image_paths, prompt_text):
    """
    Use Claude to extract text from multiple images.

    :param api_key: Your Anthropic API key
    :param image_paths: List of paths to image files
    :param prompt_text: Text prompt for Claude
    :return: Extracted text from the images
    """
    # Initialize the Anthropic client
    client = Anthropic(api_key=api_key)

    try:
        # Create content array
        content = []

        # Add each image to the content array
        image_added = False
        for image_path in image_paths:
            # Determine media type based on file extension
            media_type = "image/jpeg"  # Default
            if image_path.lower().endswith(".png"):
                media_type = "image/png"
            elif image_path.lower().endswith(".gif"):
                media_type = "image/gif"
            elif image_path.lower().endswith(".webp"):
                 media_type = "image/webp" # Add webp support


            # Encode the image
            logging.info(f"Encoding image: {os.path.basename(image_path)}")
            base64_image = encode_image(image_path)

            if base64_image:
                # Add image to content array
                content.append({
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": media_type,
                        "data": base64_image
                    }
                })
                image_added = True
            else:
                logging.warning(f"Skipping image due to encoding error: {os.path.basename(image_path)}")

        if not image_added:
            logging.error("No images could be successfully encoded and added.")
            return None

        # Add text prompt at the end
        content.append({
            "type": "text",
            "text": prompt_text
        })

        # Send request to Claude
        logging.info("Sending request to Claude API...")
        response = client.messages.create(
            model="claude-3-haiku-20240307", # Using Haiku for speed/cost balance
            # model="claude-3-opus-20240229", # Opus might give better results but is slower/more expensive
            max_tokens=4000, # Increased max_tokens slightly
            messages=[
                {
                    "role": "user",
                    "content": content
                }
            ]
        )

        # Return the extracted text
        extracted_text = response.content[0].text
        logging.info(f"Received response from Claude. Content length: {len(extracted_text)} characters.")
        # Log first few lines of response for debugging
        # logging.debug("Claude Response Head:\n" + "\n".join(extracted_text.split('\n')[:5]))
        return extracted_text

    except Exception as e:
        logging.error(f"An error occurred during Claude API call: {e}", exc_info=True) # Log traceback
        return None

def parse_menu_items(extracted_text):
    """
    Parse menu items, prices, and categories from extracted text.

    :param extracted_text: Text extracted from the image by Claude
    :return: Dictionary of menu items {name: {'price': float, 'category': str}}, plus confidence scores {name: float}
    """
    menu_items = {}
    confidence_scores = {}

    # Basic check if extraction returned anything
    if not extracted_text or not extracted_text.strip():
        logging.warning("Received empty or null text from extraction.")
        return {}, {}

    # Split the text into lines
    lines = extracted_text.strip().split('\n')
    total_non_empty_lines = sum(1 for line in lines if line.strip())
    successful_extractions = 0
    unclear_items = 0
    default_category_items = 0

    logging.info(f"Parsing {total_non_empty_lines} non-empty lines from Claude output.")

    # --- Refined Regex Patterns ---
    # Price pattern: Handles various currency symbols (optional), commas, and decimals
    # Makes currency symbol and spacing optional. Allows integer prices.
    price_pattern = r'(?:[$€£¥]\s*|\b)(\d+(?:,\d{3})*(?:\.\d{1,2})?)\b'
    # Regex to find item name, price, and category (assumes format: NAME PRICE CATEGORY)
    # It tries to be flexible with spacing and potential currency symbols.
    # Group 1: Item Name (non-greedy)
    # Group 2: Price (using price_pattern logic)
    # Group 3: Category (rest of the line)
    # This regex is complex and might need tuning based on Claude's actual output format.
    # Let's try a simpler approach first: find price, then split.

    for line_num, line in enumerate(lines):
        line = line.strip()
        # Skip empty lines or potential headers/footers from Claude
        if not line or line.startswith("Here are the items") or line.startswith("---"):
            continue

        line_confidence = 1.0  # Start with full confidence

        # Check for uncertainty indicators in the text (global line check)
        uncertainty_phrases = ["unclear", "can't make out", "illegible", "not visible", "hard to read", "possibly", "maybe", "appears to be"]
        is_unclear = False
        for phrase in uncertainty_phrases:
            if phrase in line.lower():
                line_confidence *= 0.6  # Reduce confidence if uncertainty is indicated globally
                is_unclear = True
                # break # Found one, no need to check others for this line

        # 1. Find the price first (more reliable anchor)
        price_match = re.search(price_pattern, line)

        if price_match:
            # Extract the price and convert to float
            price_str = price_match.group(1).replace(',', '')
            try:
                price = float(price_str)
            except ValueError:
                logging.warning(f"Line {line_num+1}: Could not convert price '{price_str}' to float. Skipping line: '{line}'")
                continue

            # 2. Extract Item Name (everything before the price match)
            item_name = line[:price_match.start()].strip()
            # Clean up common trailing characters before price
            item_name = re.sub(r'[.…\-_*\s]+$', '', item_name).strip()

            # 3. Extract Category (everything after the price match)
            category_name = line[price_match.end():].strip()
            # Clean up common leading characters after price
            category_name = re.sub(r'^[.…\-_*\s]+', '', category_name).strip()

            # --- Data Cleaning and Validation ---
            # Handle empty item name (likely parsing error or header)
            if not item_name:
                 logging.warning(f"Line {line_num+1}: Extracted empty item name. Skipping line: '{line}'")
                 continue

            # Handle empty or default category
            if not category_name or category_name.lower() in ["unclear", "unknown", "n/a", "none", "-", "--"]:
                category_name = "Uncategorized" # Standardize default
                line_confidence *= 0.8 # Slightly reduce confidence if category was unclear/missing
                default_category_items += 1

            # Remove any "(unclear)" tags added by Claude from name/category
            item_name = item_name.replace("(unclear)", "").strip()
            category_name = category_name.replace("(unclear)", "").strip()

            # If the global 'unclear' flag was set, mark item count
            if is_unclear:
                unclear_items += 1


            # Check name/category quality (very short strings might be noise)
            if len(item_name) < 3:
                line_confidence *= 0.8
            if len(category_name) < 3 and category_name != "Uncategorized": # Allow short defaults
                 line_confidence *= 0.9 # Less penalty for short category

            # Check price reasonableness
            if price <= 0 or price > 1000: # Adjusted range slightly
                line_confidence *= 0.7

            # Store the item
            successful_extractions += 1
            item_key = item_name # Use name as the key

            # Handle duplicate item names (append index if price/category differ)
            if item_key in menu_items:
                 existing_item = menu_items[item_key]
                 # Only create variant if price OR category is different
                 if existing_item['price'] != price or existing_item['category'] != category_name:
                     count = 1
                     new_key = f"{item_name} ({count})"
                     while new_key in menu_items:
                         count += 1
                         new_key = f"{item_name} ({count})"
                     item_key = new_key
                     logging.info(f"Duplicate item name '{item_name}' found with different price/category. Storing as '{item_key}'.")
                 else:
                     # Exact duplicate, maybe increase confidence slightly? Or just skip. Let's skip.
                     logging.info(f"Exact duplicate item found: '{item_name}'. Skipping.")
                     successful_extractions -= 1 # Decrement success counter as we skipped it
                     continue # Skip to next line

            menu_items[item_key] = {'price': price, 'category': category_name}
            confidence_scores[item_key] = max(0.0, min(1.0, line_confidence)) # Clamp confidence 0-1

        else:
            # Line did not contain a recognizable price pattern
            logging.warning(f"Line {line_num+1}: No valid price pattern found. Skipping line: '{line}'")

    # Calculate overall extraction quality metrics
    extraction_rate = successful_extractions / total_non_empty_lines if total_non_empty_lines > 0 else 0
    avg_confidence = sum(confidence_scores.values()) / len(confidence_scores) if confidence_scores else 0

    # Log the extraction metrics
    logging.info(f"Parsing complete. Successfully extracted {successful_extractions} items.")
    logging.info(f"Extraction rate: {extraction_rate:.1%} of non-empty lines.")
    logging.info(f"Items marked unclear by Claude or parser: {unclear_items}")
    logging.info(f"Items assigned default 'Uncategorized' category: {default_category_items}")
    logging.info(f"Average confidence score for extracted items: {avg_confidence:.1%}")

    return menu_items, confidence_scores

def categorize_menu_items(menu_items, num_categories=3):
    """
    Categorize menu items based on their price into groups like A, B, C.

    :param menu_items: Dictionary {name: {'price': float, 'category': str}}
    :param num_categories: Number of price categories (A, B, C...)
    :return: Dictionary {price_category: [{'name': str, 'price': float, 'category': str}, ...]}
    """
    if not menu_items:
        logging.warning("No menu items provided for price categorization.")
        return {}

    # Prepare items list: [{'name': name, 'price': price, 'category': category}, ...]
    items_list = [{'name': name, **data} for name, data in menu_items.items()]

    # Sort items by price (descending)
    sorted_items_list = sorted(items_list, key=lambda x: x['price'], reverse=True)

    # Determine price ranges
    prices = [item['price'] for item in sorted_items_list]
    if not prices: # Should not happen if menu_items is not empty, but safety check
        return {}

    max_price = max(prices)
    min_price = min(prices)
    price_range = max_price - min_price

    logging.info(f"\nPrice range for categorization: ${min_price:.2f} to ${max_price:.2f} (Spread: ${price_range:.2f})")

    # Create categories based on price ranges
    # Result structure: {'A': [item_dict1, item_dict2], 'B': [...]}
    categories = defaultdict(list)

    # Handle edge cases: zero range or only one category requested
    if num_categories <= 1 or price_range == 0:
        logging.info("Assigning all items to Category A (single category requested or zero price range).")
        categories['A'].extend(sorted_items_list)
        return dict(categories) # Convert back to dict from defaultdict

    # --- Adaptive Categorization Logic (from original code) ---
    # Decide whether to use equal price range division or percentile-based division
    large_range_threshold = 3.0 # Use adaptive if range > 3x min price
    use_adaptive = price_range > min_price * large_range_threshold and min_price > 0

    if use_adaptive:
        logging.info(f"Large price range detected (range/min > {large_range_threshold:.1f}) - using adaptive (percentile-based) categorization.")

        num_items = len(sorted_items_list)
        category_boundaries = [] # List of lower bounds for categories B, C, D...

        # Calculate boundary prices based on item count percentiles
        for i in range(1, num_categories):
            idx = int((i * num_items) / num_categories)
            # Use the price of the item at the boundary index as the threshold
            # Ensure index is valid
            if 0 <= idx < num_items:
                 # The boundary is the price *below which* items fall into the next category
                 # So, category A is >= boundary[0], B is < boundary[0] and >= boundary[1], etc.
                 boundary_price = sorted_items_list[idx]['price']
                 # Add a small epsilon if needed to handle exact matches at boundaries consistently
                 # Let's adjust logic: boundary price IS the lower limit for the higher category
                 category_boundaries.append(boundary_price)
            else:
                 # Should not happen with valid indices, but handle gracefully
                 logging.warning(f"Could not determine boundary price for category {chr(65 + i)}")
                 # Use the previous boundary or min price as fallback
                 category_boundaries.append(category_boundaries[-1] if category_boundaries else min_price)

        # Ensure boundaries are unique and sorted descending
        category_boundaries = sorted(list(set(category_boundaries)), reverse=True)

        # Add the minimum price as the effective floor for the last category
        category_boundaries.append(min_price - 0.01) # Ensure min_price items are included

        boundary_strs = [f">${b:.2f}" for b in category_boundaries[:-1]] # Don't print the floor boundary
        logging.info(f"Adaptive category price boundaries (Min price for Cat A, B, C...): {boundary_strs}")

        # Assign items to categories based on these boundaries
        for item_dict in sorted_items_list:
            item_price = item_dict['price']
            assigned = False
            # Find the first boundary the price is >= to
            for i, boundary in enumerate(category_boundaries[:-1]): # Iterate through A, B, C... boundaries
                 if item_price >= boundary:
                     category_letter = chr(65 + i)
                     categories[category_letter].append(item_dict)
                     assigned = True
                     break
            # If not assigned (should only happen for min_price items falling below last explicit boundary)
            if not assigned:
                 # Assign to the last category
                 category_letter = chr(65 + num_categories - 1)
                 categories[category_letter].append(item_dict)

    else:
        # --- Standard Equal Price Range Division ---
        logging.info("Using standard categorization (equal price range division).")
        # Avoid division by zero if num_categories is 0 or less (handled earlier, but safety)
        if num_categories <= 0: num_categories = 1
        category_range_size = price_range / num_categories

        # Print calculated category ranges
        logging.info(f"Calculated category range size: ${category_range_size:.2f}")
        for i in range(num_categories):
             cat_letter = chr(65 + i)
             cat_max = max_price - (i * category_range_size)
             cat_min = max_price - ((i + 1) * category_range_size)
             # Ensure the last category includes the minimum price exactly
             if i == num_categories - 1: cat_min = min_price
             logging.info(f"  Target range for Category {cat_letter}: ~${cat_min:.2f} - ${cat_max:.2f}")


        for item_dict in sorted_items_list:
            item_price = item_dict['price']
            # Determine category index based on price position within the total range
            # Handle max_price edge case: should be in Category A (index 0)
            if item_price == max_price:
                category_index = 0
            # Avoid division by zero if category_range_size is 0 (handled earlier, but safety)
            elif category_range_size > 0:
                 # Calculate how many 'ranges' down from the max price this item is
                 category_index = int((max_price - item_price) / category_range_size)
                 # Clamp index to valid range [0, num_categories - 1]
                 category_index = max(0, min(num_categories - 1, category_index))
            else: # category_range_size is 0 (all prices same) -> should be handled by initial check
                 category_index = 0

            category_letter = chr(65 + category_index)
            categories[category_letter].append(item_dict)

    # Log the distribution of items across final categories
    logging.info("\n--- Final Item Distribution by Price Category ---")
    for category_letter, items in sorted(categories.items()):
        category_min_price = min(item['price'] for item in items) if items else 0
        category_max_price = max(item['price'] for item in items) if items else 0
        logging.info(f"Category {category_letter}: {len(items)} items. Actual price range: ${category_min_price:.2f} - ${category_max_price:.2f}")

    return dict(categories) # Convert back to regular dict


def process_menu_images(api_key, image_folder, num_price_categories=3):
    """
    Process all menu images in a folder, extract items (name, price, category),
    and categorize them by price tiers (A, B, C...).

    :param api_key: Your Anthropic API key
    :param image_folder: Path to folder containing menu images
    :param num_price_categories: Number of price categories (A, B, C...)
    :return: Price-categorized menu items dict, and overall confidence scores dict
    """
    # List all image files in the folder
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.webp'] # Added webp
    image_paths = []

    if not os.path.isdir(image_folder):
        logging.error(f"Image folder not found or is not a directory: {image_folder}")
        return {}, {}

    for file in os.listdir(image_folder):
        if any(file.lower().endswith(ext) for ext in image_extensions):
            image_paths.append(os.path.join(image_folder, file))

    if not image_paths:
        logging.error(f"No supported image files ({', '.join(image_extensions)}) found in {image_folder}")
        return {}, {}

    logging.info(f"Found {len(image_paths)} images to process in folder: {image_folder}")

    # Store all items extracted across all images before final categorization
    # Using the structure: {name: {'price': float, 'category': str}}
    all_menu_items_combined = {}
    all_confidence_scores_combined = {}
    image_extraction_counts = []

    # --- Updated Prompt for Claude ---
    prompt_text = f"""
    Please extract all menu items from the provided image(s).
    For each distinct menu item found, provide the following information on a single line:
    1. The complete Item Name.
    2. The exact Price (including currency symbol like $ or € if visible, otherwise just the number).
    3. The item's Category (e.g., Appetizer, Main Course, Dessert, Beverage, Side Dish).

    Format each item STRICTLY as:
    Item Name $Price Category

    Examples:
    Classic Caesar Salad $12.99 Appetizer
    Grilled Salmon Fillet $24.50 Main Course
    New York Cheesecake $8.00 Dessert
    Iced Tea $3.50 Beverage
    French Fries $5.00 Side Dish

    IMPORTANT INSTRUCTIONS:
    - List EVERY visible menu item, even if unsure about details.
    - If you cannot clearly determine the Item Name, Price, or Category, use the placeholder "(unclear)" for that specific part. For example: "House Special (unclear) $16.99 Main Course" or "Spicy Tuna Roll $15.00 (unclear)". If the category is totally unknown, use "Uncategorized".
    - Ensure the price is extracted accurately.
    - Do NOT add any introductory text, explanations, summaries, or formatting like bullet points or markdown. Only output the list of items in the specified format, one item per line.
    - If an item seems to span multiple lines in the menu, combine it into a single logical item name if possible.
    - Pay attention to sections or headers in the menu image to help determine the category.
    """

    # Process images (can be done one-by-one or batched if API/model supports it well)
    # Let's stick to one-by-one for simplicity and robustness against single image failures.
    for idx, image_path in enumerate(image_paths):
        logging.info(f"\n--- Processing image {idx+1}/{len(image_paths)}: {os.path.basename(image_path)} ---")

        extracted_text = extract_text_from_image(api_key, [image_path], prompt_text)

        if extracted_text:
            logging.info(f"Text extraction successful for {os.path.basename(image_path)}. Parsing content...")

            # Parse menu items (name, price, category) from this image's text
            menu_items_single, confidence_scores_single = parse_menu_items(extracted_text)
            item_count_single = len(menu_items_single)
            logging.info(f"Parsed {item_count_single} items from {os.path.basename(image_path)}.")
            image_extraction_counts.append((os.path.basename(image_path), item_count_single))

            # Merge items from this image into the combined dictionary
            for item_name, item_data in menu_items_single.items():
                confidence = confidence_scores_single.get(item_name, 0.5) # Default confidence if missing

                # Check if item already exists in combined list
                if item_name in all_menu_items_combined:
                    existing_data = all_menu_items_combined[item_name]
                    # If price or category differs, create a variant
                    if existing_data['price'] != item_data['price'] or existing_data['category'] != item_data['category']:
                        variant_count = 1
                        variant_name = f"{item_name} (Img {idx+1})" # Add image source to variant name
                        while variant_name in all_menu_items_combined:
                             variant_count += 1
                             variant_name = f"{item_name} (Img {idx+1} Var {variant_count})"

                        logging.warning(f"Item '{item_name}' from {os.path.basename(image_path)} differs from previous entry. Storing as '{variant_name}'. "
                                        f"Old: P={existing_data['price']}, C='{existing_data['category']}'. New: P={item_data['price']}, C='{item_data['category']}'.")
                        all_menu_items_combined[variant_name] = item_data
                        all_confidence_scores_combined[variant_name] = confidence
                    else:
                         # Exact same item found again, potentially update confidence if higher?
                         # For now, let's keep the first encountered confidence.
                         logging.info(f"Item '{item_name}' is an exact duplicate from another image. Skipping.")
                else:
                    # New item, add it
                    all_menu_items_combined[item_name] = item_data
                    all_confidence_scores_combined[item_name] = confidence

            logging.info(f"Combined total unique items so far: {len(all_menu_items_combined)}")
        else:
            logging.error(f"Failed to extract text from {os.path.basename(image_path)}. Skipping this image.")
            image_extraction_counts.append((os.path.basename(image_path), 0)) # Record failure

    # --- Post-Processing and Categorization ---
    logging.info(f"\n===== Image Processing Summary =====")
    total_items_extracted = sum(count for _, count in image_extraction_counts)
    logging.info(f"Total items parsed across all images (before deduplication/variants): {total_items_extracted}")
    logging.info(f"Final unique items/variants stored: {len(all_menu_items_combined)}")

    # Print extraction performance by image
    logging.info("\nItems Parsed Per Image:")
    for image_name, item_count in image_extraction_counts:
        logging.info(f"  {image_name}: {item_count} items")

    # Calculate overall average confidence score
    if all_confidence_scores_combined:
        avg_confidence = sum(all_confidence_scores_combined.values()) / len(all_confidence_scores_combined)
        logging.info(f"Overall average extraction confidence: {avg_confidence:.1%}")
    else:
         logging.info("No confidence scores available.")


    if not all_menu_items_combined:
        logging.error("No menu items were successfully extracted from any images.")
        return {}, {}

    # --- Dynamic Adjustment of Price Categories ---
    # Adjust num_categories based on item count for better distribution
    min_items_per_category = 3 # Aim for at least 3 items per category if possible
    max_possible_categories = len(all_menu_items_combined) // min_items_per_category
    adjusted_categories = max(1, min(num_price_categories, max_possible_categories))

    if adjusted_categories != num_price_categories:
        logging.warning(f"Adjusting number of price categories from {num_price_categories} to {adjusted_categories} "
                      f"based on item count ({len(all_menu_items_combined)}) and target minimum items per category ({min_items_per_category}).")
        num_price_categories = adjusted_categories
    else:
         logging.info(f"Using requested number of price categories: {num_price_categories}")

    # Categorize the combined menu items by price tiers (A, B, C...)
    logging.info("\n--- Categorizing All Extracted Items by Price ---")
    final_price_categories = categorize_menu_items(all_menu_items_combined, num_price_categories)

    return final_price_categories, all_confidence_scores_combined

def main():
    # --- Configuration ---
    # IMPORTANT: Replace with your actual Anthropic API key
    # Consider using environment variables for API keys in production
    API_KEY = os.environ.get('ANTHROPIC_API_KEY') # Looks for env var first

    if not API_KEY:
        logging.error("CRITICAL: Anthropic API key not found. Set the ANTHROPIC_API_KEY environment variable or replace 'YOUR_API_KEY_HERE' in the script.")
        return

    # Path to your folder containing menu images
    IMAGE_FOLDER = './test_menus/test_menu2' # Make sure this folder exists and contains images

    # Target number of *price* categories (e.g., A, B, C, D for 4 categories)
    NUM_PRICE_CATEGORIES = 4

    # Output file name
    OUTPUT_FILENAME = "categorized_menu_items.txt"
    # --- End Configuration ---


    logging.info("===== MENU ITEM EXTRACTION AND PRICE CATEGORIZATION =====")
    logging.info(f"Processing menu images from folder: {os.path.abspath(IMAGE_FOLDER)}")
    logging.info(f"Target number of price categories: {NUM_PRICE_CATEGORIES}")

    # Create image folder if it doesn't exist (useful for first run)
    if not os.path.exists(IMAGE_FOLDER):
        try:
            os.makedirs(IMAGE_FOLDER)
            logging.info(f"Created image folder: {IMAGE_FOLDER}")
            logging.warning("Image folder was created, but it's currently empty. Please add menu images to it.")
            return # Stop execution if folder was just created empty
        except OSError as e:
            logging.error(f"Failed to create image folder {IMAGE_FOLDER}: {e}")
            return


    # Process menu images
    price_categories, confidence_scores = process_menu_images(API_KEY, IMAGE_FOLDER, NUM_PRICE_CATEGORIES)

    if price_categories:
        # --- Output Results ---
        logging.info("\n========== FINAL MENU ITEMS BY PRICE CATEGORY ==========")
        # Print to console
        for category_letter, items in sorted(price_categories.items()):
            if items:
                # Sort items within the category by price (desc) for printing
                items_sorted = sorted(items, key=lambda x: x['price'], reverse=True)
                category_min_price = items_sorted[-1]['price']
                category_max_price = items_sorted[0]['price']
                print(f"\n----- Category {category_letter} (${category_min_price:.2f} - ${category_max_price:.2f}) -----")
                print(f"{len(items)} items:")

                for item_dict in items_sorted:
                    # Display Name, Price, and the SEMANTIC Category extracted by Claude
                    print(f"  ${item_dict['price']:.2f} - {item_dict['name']} ({item_dict['category']})") # Include semantic category in print

        # Save categorized items to a file
        try:
            output_path = os.path.abspath(OUTPUT_FILENAME)
            logging.info(f"\nSaving results to: {output_path}")
            with open(output_path, "w", encoding='utf-8') as f:
                f.write("===== MENU ITEMS BY PRICE CATEGORY (Generated by Script) =====\n")
                f.write(f"Processed images from: {os.path.abspath(IMAGE_FOLDER)}\n")
                # Optionally add confidence score info here if desired
                # avg_conf = (sum(confidence_scores.values()) / len(confidence_scores)) * 100 if confidence_scores else 0
                # f.write(f"Average Extraction Confidence: {avg_conf:.1f}%\n")

                for category_letter, items in sorted(price_categories.items()):
                    if items:
                        # Sort items within the category by price (desc) for writing
                        items_sorted = sorted(items, key=lambda x: x['price'], reverse=True)
                        category_min_price = items_sorted[-1]['price']
                        category_max_price = items_sorted[0]['price']
                        f.write(f"\n----- Category {category_letter} (${category_min_price:.2f} - ${category_max_price:.2f}) -----\n")

                        for item_dict in items_sorted:
                            # Write in the format expected by the *other* script,
                            # BUT let's include the semantic category in parentheses.
                            # The other script's parser might need adjustment if it can't handle this.
                            f.write(f"  ${item_dict['price']:.2f} - {item_dict['name']} ({item_dict['category']})\n")
            logging.info(f"Results successfully saved to {OUTPUT_FILENAME}")
        except IOError as e:
            logging.error(f"Error saving results to file '{OUTPUT_FILENAME}': {e}")
        except Exception as e:
            logging.error(f"An unexpected error occurred during file writing: {e}", exc_info=True)
    else:
        logging.error("Processing finished, but no menu items were found or categorized.")

if __name__ == "__main__":
    main()

2025-04-25 22:33:04,057 - INFO - ===== MENU ITEM EXTRACTION AND PRICE CATEGORIZATION =====
2025-04-25 22:33:04,062 - INFO - Processing menu images from folder: /Users/Cwkf_89/Documents/GitHub/hungryhub_automation/test_menus/test_menu2
2025-04-25 22:33:04,067 - INFO - Target number of price categories: 4
2025-04-25 22:33:04,093 - INFO - Found 11 images to process in folder: ./test_menus/test_menu2
2025-04-25 22:33:04,094 - INFO - 
--- Processing image 1/11: 24-04-26_กับภูเก็ต_Menu book A3_Revise02_AW Only_page-0008.jpg ---
2025-04-25 22:33:04,108 - INFO - Encoding image: 24-04-26_กับภูเก็ต_Menu book A3_Revise02_AW Only_page-0008.jpg
2025-04-25 22:33:04,115 - INFO - Sending request to Claude API...
2025-04-25 22:33:08,014 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-04-25 22:33:08,016 - INFO - Received response from Claude. Content length: 439 characters.
2025-04-25 22:33:08,017 - INFO - Text extraction successful for 24-04-26_กับภูเก็ต_Menu bo


----- Category A ($268.00 - $495.00) -----
22 items:
  $495.00 - Sweet and Sour Soup Phuket Style with Trevally Fish (1) (THB Hot and Spicy Soup)
  $490.00 - Deep Fried Prawns' Meat with Fried Curry Paste ($590 Stir-Fried and Deep-Fried Dishes)
  $490.00 - Fried Trevally Fish Meat Mixed with Curry Paste (/$590.00 Main Course)
  $455.00 - Sweet and Sour Soup Phuket Style with Trevally Fish (/599 THB Hot and Spicy Soup)
  $455.00 - Southern Curry with Trevally Fish without Coconut Milk (1) (/$599.00 Main Course)
  $450.00 - Hot and Spicy Soup (1) (Main Course)
  $450.00 - Chilli Paste Recommended Dish ฉีดเนื้อพริกสด ฉันต้านาน (Chilli Paste Dishes)
  $439.00 - Stir Fried and Deep-Fried Dishes (Baht / 540 Baht Uncategorized)
  $400.00 - Sliced Fish Meat Marinated with Tamarind Juice,Lemon Grass, Galangal, Basil and Mint ปลาซั่มย้างสมพะพริก (/550 บาท Specialty Dish)
  $390.00 - Stir Fried Slender Fish Egg with Bitter Bean (Main Course)
  $350.00 - Hot and Spicy Soup (Img 6) (Main Course)
 

<h2>Bundle Generation</h2>

In [5]:
import base64
import os
from google import genai
from google.genai import types
import json
import re

from dotenv import load_dotenv
load_dotenv()


def generate():
    try:
        client = genai.Client(
            api_key=os.getenv("GEMINI_API_KEY"),
        )
    except Exception as e:
        print(f"Error initializing Gemini client: {e}")
        return
    
    # Check if categorized_menu_items.txt exists
    if not os.path.exists("./categorized_menu_items.txt"):
        print("Error: categorized_menu_items.txt file not found")
        return
        
    try:
        # Read the file content
        with open("./categorized_menu_items.txt", "r") as file:
            menu_content = file.read()
    except Exception as e:
        print(f"Error reading menu file: {e}")
        return
    
    # Create prompt with the file content
    prompt = f"""
    Based on these categorised menu data:
    
    {menu_content}
    
    Please create 4 UNIQUE menu bundles for a varying number of diners (1-6 diners). Each bundle must include some items from each available menu category (A, B, C, and D if present).
    
    For each menu bundle, include:
    1. The number of menu items from each category
    2. The number of diners the bundle is designed for
    3. The price per diner
    4. A suggested discounted bundle price

    Please follow this exact structure for each menu bundle:
    Suggested bundle price:
    Number of diners:
    category_portionss:
        Category A: [number of items]
        Category B: [number of items]
        Category C: [number of items]
        Category D: [number of items]
    Original bundle price:
    Discount percentage:
    Price per diner:
    
    Note: Include all four categories (A through D) in your response, even if some categories might have 0 items in certain bundles.
    """

    model = "gemini-2.5-pro-exp-03-25"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=prompt),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        response_mime_type="application/json",
        system_instruction=[
            types.Part.from_text(text="""You are a menu bundle generator, and your task is to create 4 UNIQUE menu bundles based on a given input of menu categories. Each bundle should be distinctly different from the others. For each bundle:
            
1. Include items from all available categories (A, B, C, and D)
2. Specify exactly how many portions from each category are included
3. Calculate the original price based on the actual menu prices
4. Apply a reasonable discount (10-20%)
5. Calculate the per-person price
            
Always include all four categories (A through D) in your response structure, even if some categories have 0 items or don't exist in the input data."""),
        ],
    )

    # Collect the entire response
    complete_response = ""
    
    try:
        # Get the response generator
        response_stream = client.models.generate_content_stream(
            model=model,
            contents=contents,
            config=generate_content_config,
        )
        
        # Check if response is None
        if response_stream is None:
            print("Warning: API returned None for response stream")
            # Try non-streaming version as fallback
            try:
                print("Attempting to use non-streaming API as fallback...")
                response = client.models.generate_content(
                    model=model,
                    contents=contents,
                    config=generate_content_config,
                )
                
                if response and hasattr(response, 'text'):
                    complete_response = response.text
                elif response and hasattr(response, 'parts'):
                    for part in response.parts:
                        if hasattr(part, 'text') and part.text is not None:
                            complete_response += part.text
                else:
                    print("Warning: Fallback response format is unexpected")
            except Exception as fallback_error:
                print(f"Fallback request also failed: {fallback_error}")
        else:
            # Iterate through streaming response if it's not None
            for chunk in response_stream:
                # Handle the case where chunk.text might be None
                # Get the text from different potential structures
                chunk_text = ""
                
                # Try direct text property
                if hasattr(chunk, 'text') and chunk.text is not None:
                    chunk_text = chunk.text
                # Try looking in parts
                elif hasattr(chunk, 'parts') and chunk.parts:
                    # Combine text from all parts
                    for part in chunk.parts:
                        if hasattr(part, 'text') and part.text is not None:
                            chunk_text += part.text
                # Try candidates structure
                elif hasattr(chunk, 'candidates') and chunk.candidates:
                    for candidate in chunk.candidates:
                        if hasattr(candidate, 'content'):
                            content = candidate.content
                            if hasattr(content, 'parts'):
                                for part in content.parts:
                                    if hasattr(part, 'text') and part.text is not None:
                                        chunk_text += part.text
                
                # Add to the complete response
                complete_response += chunk_text
                
                # Print to console
                if chunk_text:
                    print(chunk_text, end="")
                else:
                    print(".", end="")  # Print a dot to indicate progress for empty chunks
                
    except Exception as e:
        print(f"\n\nError during generation: {e}")
        
        # Last-ditch effort - try non-streaming API if streaming failed
        if not complete_response:
            try:
                print("Attempting to use non-streaming API as final fallback...")
                response = client.models.generate_content(
                    model=model,
                    contents=contents,
                    config=generate_content_config,
                )
                
                if hasattr(response, 'text') and response.text:
                    complete_response = response.text
                    print(f"Received {len(complete_response)} characters from fallback request")
            except Exception as fallback_error:
                print(f"Fallback request also failed: {fallback_error}")
    
    # Save the raw response for debugging
    try:
        with open("menu_bundles_raw.txt", "w") as f:
            f.write(complete_response)
    except Exception as e:
        print(f"Error saving raw response: {e}")
    
    # Parse and save as JSON
    try:
        print("\n\nParsing response as JSON...")
        
        # Check if the response is empty
        if not complete_response.strip():
            raise ValueError("Empty response received from API")
            
        # Try direct JSON parsing first
        all_valid_jsons = []
        unique_bundles = set()  # Use a set to track unique bundles
        
        try:
            # First, try parsing the entire response as a single JSON object
            json_data = json.loads(complete_response)
            if isinstance(json_data, list):
                # If it's a list, use it directly
                all_valid_jsons = json_data[:4]  # Limit to 4 bundles
            else:
                # If it's an object, check if it has a 'bundles' property
                if "bundles" in json_data and isinstance(json_data["bundles"], list):
                    json_data["bundles"] = json_data["bundles"][:4]  # Limit to 4 bundles
                all_valid_jsons = [json_data]
        except json.JSONDecodeError:
            print("Direct JSON parsing failed, trying extraction methods...")
            
            # Look for JSON objects pattern
            json_pattern = r'(\{(?:[^{}]|(?:\{(?:[^{}]|(?:\{[^{}]*\}))*\}))*\})'
            json_matches = re.findall(json_pattern, complete_response)
            
            if json_matches:
                # Try each potential JSON match
                potential_jsons = sorted(json_matches, key=len, reverse=True)
                
                for potential_json in potential_jsons:
                    try:
                        json_data = json.loads(potential_json)
                        # Convert to string for comparison to check uniqueness
                        json_str = json.dumps(json_data, sort_keys=True)
                        if json_str not in unique_bundles:
                            unique_bundles.add(json_str)
                            all_valid_jsons.append(json_data)
                    except json.JSONDecodeError:
                        continue
            
            # Also try code blocks
            code_block_matches = re.findall(r'```(?:json)?(.*?)```', complete_response, re.DOTALL)
            for code_block in code_block_matches:
                try:
                    json_data = json.loads(code_block.strip())
                    # Check uniqueness again
                    json_str = json.dumps(json_data, sort_keys=True)
                    if json_str not in unique_bundles:
                        unique_bundles.add(json_str)
                        all_valid_jsons.append(json_data)
                except json.JSONDecodeError:
                    continue
        
        # Ensure we have only 4 bundles maximum
        if len(all_valid_jsons) > 4:
            all_valid_jsons = all_valid_jsons[:4]
        
        # Save all valid JSONs to a single file
        if all_valid_jsons:
            with open("menu_bundles.json", "w") as json_file:
                json.dump(all_valid_jsons, json_file, indent=4)
            print(f"Saved {len(all_valid_jsons)} unique menu bundles to menu_bundles.json")
        else:
            print("No valid JSONs found in the response")
            
    except Exception as e:
        print(f"Error: Failed to parse response as JSON: {e}")
        print("Please check menu_bundles_raw.txt to see the actual response.")

if __name__ == "__main__":
    generate()

2025-04-25 22:34:18,304 - INFO - AFC is enabled with max remote calls: 10.
2025-04-25 22:34:18,305 - INFO - AFC remote call 1 is done.
2025-04-25 22:34:43,818 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro-exp-03-25:streamGenerateContent?alt=sse "HTTP/1.1 200 OK"


[
  {
    "bundle_name": "Solo Feast",
    "suggested_bundle_price": 593.00,
    "number_of_diners": 1,
    "category_portions": {
      "Category A": 1,
      "Category B": 1,
      "Category C": 1,
      "Category D": 1
    },
    "original_bundle_price": 674.00,
    "discount_percentage": 12.02,
    "price_per_diner": 593.00
  },
  {
    "bundle_name": "Couple's Delight",
    "suggested_bundle_price": 973.00,
    "number_of_diners": 2,
    "category_portions": {
      "Category A": 1,
      "Category B": 1,
      "Category C": 1,
      "Category D": 3
    },
    "original_bundle_price": 1145.00,
    "discount_percentage": 15.02,
    "price_per_diner": 486.50
  },
  {
    "bundle_name": "Phuket Flavors Sampler",
    "suggested_bundle_price": 1440.00,
    "number_of_diners": 4,
    "category_portions": {
      "Category A": 2,
      "Category B": 2,
      "Category C": 2,
      "Category D": 6
    },
    "original_bundle_price": 1756.00,
    "discount_percentage": 18.00,
    "price_pe

<h2>Excel Generation</h2>

In [6]:
import json
import openpyxl
import os
import re # Import the regular expression module
from openpyxl.styles import PatternFill, Font, Alignment, Border, Side
from openpyxl.utils import get_column_letter
from openpyxl.cell.cell import MergedCell
import logging # Use logging for better messages

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


def parse_menu_items(file_path='./categorized_menu_items.txt'):
    """
    Parse the categorized menu items text file, expecting format:
    $Price - Item Name (Category)

    Args:
        file_path (str): Path to the text file containing categorized menu items

    Returns:
        dict: Dictionary with price categories ('Category A', etc.) as keys
            and lists of menu item dicts ({'name': str, 'price': float, 'category': str}) as values.
    """
    # These keys represent the PRICE BANDS (A, B, C, D) determined by the other script
    menu_items_by_price_band = {
        "Category A": [],
        "Category B": [],
        "Category C": [],
        "Category D": []
    }

    absolute_path = os.path.abspath(file_path)
    logging.info(f"Attempting to read menu items from: {absolute_path}")

    try:
        if not os.path.exists(absolute_path):
            raise FileNotFoundError(f"File not found at {absolute_path}")

        current_price_band = None # e.g., "Category A", "Category B"
        items_parsed_count = 0

        # Regex to capture Name and Category from "Name (Category)" format
        # Allows for spaces in name and category. Handles optional space before parenthesis.
        # Group 1: Item Name (non-greedy)
        # Group 2: Semantic Category (inside parentheses)
        name_category_pattern = re.compile(r'^(.*?)\s*\(([^)]+)\)$')

        with open(absolute_path, 'r', encoding='utf-8') as file:
            for line_num, line in enumerate(file, 1):
                line = line.strip()

                # Skip empty lines or separator/header lines
                if not line or line.startswith('====='):
                    continue

                # Check if this is a PRICE BAND header line (e.g., ----- Category A -----)
                if line.startswith('----- Category'):
                    try:
                        # Extract price band character (A, B, C, or D)
                        band_char = line.split('Category ')[1].strip()[0]
                        current_price_band = f"Category {band_char}"
                        if current_price_band not in menu_items_by_price_band:
                            logging.warning(f"Line {line_num}: Found unexpected price band '{current_price_band}'. Ignoring section.")
                            current_price_band = None # Reset if band is not A, B, C, or D
                        else:
                            logging.info(f"Line {line_num}: Found Price Band Header '{current_price_band}'")
                    except IndexError:
                        logging.warning(f"Line {line_num}: Malformed price band header: '{line}'. Ignoring.")
                        current_price_band = None

                # Check if this is a menu item line (starts with '$') and we are inside a valid price band
                elif line.startswith('$') and current_price_band:
                    try:
                        # Split the line at the first " - " to separate price from name+category
                        parts = line.split(' - ', 1)
                        if len(parts) == 2:
                            price_str = parts[0].strip()
                            name_and_category_str = parts[1].strip()

                            # Convert price string to number
                            price_value = float(price_str.replace('$', '').replace(',', ''))

                            # Now, parse the "Item Name (Category)" part
                            match = name_category_pattern.match(name_and_category_str)
                            if match:
                                item_name = match.group(1).strip() # Extract name
                                semantic_category = match.group(2).strip() # Extract category from parentheses
                            else:
                                # If format doesn't match "Name (Category)", treat whole string as name
                                # and assign a default category. Log a warning.
                                logging.warning(f"Line {line_num}: Item format mismatch. Expected '$Price - Name (Category)', got '$Price - {name_and_category_str}'. Using full string as name and 'Uncategorized'.")
                                item_name = name_and_category_str
                                semantic_category = "Uncategorized" # Default semantic category

                            # Append the item data (including semantic category) to the current price band list
                            menu_items_by_price_band[current_price_band].append({
                                'name': item_name,
                                'price': price_value,
                                'category': semantic_category  # Store the extracted semantic category
                            })
                            items_parsed_count += 1
                        else:
                            logging.warning(f"Line {line_num}: Malformed item line. Expected format '$Price - Name (Category)', got: '{line}'.")
                    except ValueError:
                        logging.warning(f"Line {line_num}: Could not parse price from '{price_str}'. Skipping item: '{line}'.")
                    except Exception as e:
                        logging.error(f"Line {line_num}: Error processing item line: '{line}'. Error: {e}", exc_info=True)
                elif current_price_band and line: # If inside a band but line doesn't start with '$'
                    logging.info(f"Line {line_num}: Skipping non-item line within {current_price_band}: '{line}'")


        if items_parsed_count > 0:
            logging.info(f"Successfully parsed {items_parsed_count} menu items from {file_path}")
        else:
            logging.warning(f"No menu items were successfully parsed from {file_path}. Check file format or content.")

    except FileNotFoundError as e:
        logging.error(f"{e}")
        logging.warning("Using default menu items because the file could not be read.")
        # Add default items WITH the 'category' key
        menu_items_by_price_band["Category A"] = [
            {"name": "Default BBQ ribs", "price": 700.0, "category": "Main Course"},
            {"name": "Default Chimichurri steak", "price": 590.0, "category": "Main Course"},
        ]
        menu_items_by_price_band["Category B"] = [
            {"name": "Default Milk Shakes", "price": 190.0, "category": "Beverage"},
        ]
        menu_items_by_price_band["Category C"] = [
            {"name": "Default Strawberry blast", "price": 100.0, "category": "Beverage"},
        ]
        menu_items_by_price_band["Category D"] = [
            {"name": "Default Extra shot", "price": 25.0, "category": "Add-on"},
        ]
    except Exception as e:
        logging.error(f"An unexpected error occurred while reading menu items file: {e}", exc_info=True)
        logging.warning("Using default menu items due to the error.")
        # Populate with defaults including category key
        menu_items_by_price_band["Category A"] = [{"name": "Error Default A", "price": 1.0, "category": "Error"}]
        menu_items_by_price_band["Category B"] = [{"name": "Error Default B", "price": 1.0, "category": "Error"}]
        menu_items_by_price_band["Category C"] = [{"name": "Error Default C", "price": 1.0, "category": "Error"}]
        menu_items_by_price_band["Category D"] = [{"name": "Error Default D", "price": 1.0, "category": "Error"}]


    # Print summary of parsed items
    logging.info("--- Parsed Menu Items Summary (by Price Band) ---")
    for price_band, items in menu_items_by_price_band.items():
        logging.info(f"  {price_band}: {len(items)} items")
        # Optional: Print first few items for verification
        # for item in items[:2]:
        #      logging.info(f"    - {item['name']} (${item['price']}) ({item['category']})")
        # if len(items) > 2:
        #      logging.info("    - ...")
    logging.info("-------------------------------------------------")

    return menu_items_by_price_band # Return the dictionary structured by price bands


def create_menu_sheet(workbook, menu_items_by_price_band):
    """
    Creates the 'Menu Items' sheet in the workbook.
    Column A ('Category') now uses the semantic category parsed from the file.

    Args:
        workbook: The openpyxl workbook to modify
        menu_items_by_price_band: Dictionary structured by price bands ('Category A', 'B', etc.),
                                containing lists of item dictionaries
                                {'name': str, 'price': float, 'category': str}.
    """
    menu_sheet = workbook.create_sheet(title='Menu Items')

    # --- Styles (same as before) ---
    red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
    light_yellow_fill = PatternFill(start_color="FFFFD4", end_color="FFFFD4", fill_type="solid")
    white_font = Font(color="FFFFFF", bold=True, size=14)
    bold_font = Font(bold=True)
    center_aligned = Alignment(horizontal='center', vertical='center', wrap_text=True)
    left_aligned = Alignment(horizontal='left', vertical='center', wrap_text=True)
    border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))

    # --- Sheet Headers and Setup (same as before) ---
    menu_sheet.merge_cells('A1:E1')
    header_cell = menu_sheet['A1']
    header_cell.value = "Hungry Hub Menu Sections"
    header_cell.fill = red_fill; header_cell.font = white_font; header_cell.alignment = center_aligned
    menu_sheet.row_dimensions[1].height = 25

    menu_sheet['G1'] = "Formula to Calculate NET Price for Menu"
    menu_sheet['G3'] = "VAT (Extra)"; menu_sheet['I3'] = "7%"
    menu_sheet['G4'] = "Service (Extra)"; menu_sheet['I4'] = "10%"
    menu_sheet['G6'] = "You can Adjust"

    column_headers = [
        "Category", # This column will now show the SEMANTIC category (Appetizer, Main, etc.)
        "Menu Name (English)",
        "Description (English or Thai) or Menu Name (Thai)", # Keeping as placeholder
        "Menu Price",
        "Price (NET) (Formula)"
    ]
    for col, header in enumerate(column_headers, start=1):
        cell = menu_sheet.cell(row=2, column=col)
        cell.value = header; cell.font = bold_font; cell.alignment = center_aligned; cell.border = border
    menu_sheet.row_dimensions[2].height = 20

    menu_sheet.column_dimensions['A'].width = 25
    menu_sheet.column_dimensions['B'].width = 40
    menu_sheet.column_dimensions['C'].width = 40
    menu_sheet.column_dimensions['D'].width = 15
    menu_sheet.column_dimensions['E'].width = 15

    current_row = 3

    # --- Populate Menu Items ---
    # Iterate through the PRICE BANDS (A, B, C, D)
    for price_band_key in ["Category A", "Category B", "Category C", "Category D"]:
        # Add the Price Band header row (e.g., "Category A")
        menu_sheet.merge_cells(start_row=current_row, start_column=1, end_row=current_row, end_column=5)
        top_left_cell = menu_sheet.cell(row=current_row, column=1, value=price_band_key)
        top_left_cell.fill = red_fill; top_left_cell.font = white_font; top_left_cell.alignment = center_aligned
        menu_sheet.row_dimensions[current_row].height = 22
        current_row += 1

        # Get the list of items belonging to this price band
        items_in_band = menu_items_by_price_band.get(price_band_key, [])

        if not items_in_band:
            logging.info(f"No items found for Price Band {price_band_key} to populate in the 'Menu Items' sheet.")
            # Optionally add a placeholder row in the sheet
            # menu_sheet.cell(row=current_row, column=1, value=f"No items found in {price_band_key}")
            # menu_sheet.merge_cells(start_row=current_row, start_column=1, end_row=current_row, end_column=5)
            # current_row += 1
            continue # Skip to the next price band

        # Add each menu item from the list for this band
        for item_dict in items_in_band: # item_dict is {'name': ..., 'price': ..., 'category': ...}

            # *** CHANGE HERE ***
            # Column A: Use the actual SEMANTIC category parsed from the file
            semantic_category = item_dict.get('category', 'Unknown') # Use .get for safety
            cell = menu_sheet.cell(row=current_row, column=1, value=semantic_category)
            cell.alignment = left_aligned
            cell.border = border

            # Column B: Menu Name (English) - From parsed data
            cell = menu_sheet.cell(row=current_row, column=2, value=item_dict.get('name', 'N/A'))
            cell.alignment = left_aligned
            cell.border = border

            # Column C: Description (Thai) - Placeholder (remains unchanged)
            cell = menu_sheet.cell(row=current_row, column=3, value="")
            cell.alignment = left_aligned
            cell.border = border

            # Column D: Menu Price - From parsed data
            price = item_dict.get('price', 0.0)
            cell = menu_sheet.cell(row=current_row, column=4, value=price)
            cell.number_format = '#,##0.00'; cell.alignment = center_aligned; cell.border = border

            # Column E: Price (NET) - Calculated (remains unchanged)
            net_price = round(price * 1.177)
            cell = menu_sheet.cell(row=current_row, column=5, value=net_price)
            cell.number_format = '#,##0'; cell.alignment = center_aligned; cell.border = border

            # Optional: Alternating row fill
            # if current_row % 2 == 0:
            #     fill_to_apply = light_yellow_fill
            #     for col_idx in range(1, 6):
            #         menu_sheet.cell(row=current_row, column=col_idx).fill = fill_to_apply

            current_row += 1

    logging.info(f"'Menu Items' sheet created. Last populated row: {current_row - 1}")
    return menu_sheet


def create_hungry_hub_proposal(json_file='./menu_bundles.json', menu_file='./categorized_menu_items.txt', output_file='HH_Proposal_Generated.xlsx'):
    """
    Creates the main Excel proposal file with both sheets.
    Uses the updated parse_menu_items and create_menu_sheet functions.
    """
    # Step 1: Read bundle data (No changes needed here)
    try:
        json_abs_path = os.path.abspath(json_file)
        logging.info(f"Attempting to read bundles from: {json_abs_path}")
        with open(json_abs_path, 'r', encoding='utf-8') as file:
            menu_bundles = json.load(file)
        logging.info(f"Successfully loaded {len(menu_bundles)} bundles from {json_file}")
        if len(menu_bundles) < 4:
            logging.warning(f"JSON file contains only {len(menu_bundles)} bundles, template expects 4.")
    except (FileNotFoundError, json.JSONDecodeError) as e:
        logging.error(f"Error reading JSON file '{json_file}': {e}")
        logging.warning("Using default menu bundles as fallback.")
        # Default bundles definition (no changes needed)
        menu_bundles = [
            { "bundle_name": "Default A", "suggested_bundle_price": "$3000", "number_of_diners": 6, "category_portions": {"Category A": 3, "Category B": 4, "Category C": 2, "Category D": 1}, "original_bundle_price": "$3500", "discount_percentage": "15%", "price_per_diner": "$500" },
            { "bundle_name": "Default B", "suggested_bundle_price": "$4000", "number_of_diners": 8, "category_portions": {"Category A": 4, "Category B": 6, "Category C": 3, "Category D": 2}, "original_bundle_price": "$4800", "discount_percentage": "20%", "price_per_diner": "$500" },
            { "bundle_name": "Default C", "suggested_bundle_price": "$5000", "number_of_diners": 10, "category_portions": {"Category A": 5, "Category B": 8, "Category C": 4, "Category D": 3}, "original_bundle_price": "$6000", "discount_percentage": "17%", "price_per_diner": "$500" },
            { "bundle_name": "Default D", "suggested_bundle_price": "$6000", "number_of_diners": 12, "category_portions": {"Category A": 6, "Category B": 10, "Category C": 5, "Category D": 4}, "original_bundle_price": "$7200", "discount_percentage": "17%", "price_per_diner": "$500" }
        ]
        menu_bundles = menu_bundles[:4] # Ensure exactly 4


    # *** Step 2: Parse menu items using the UPDATED parser ***
    # This will now return the data structure including the semantic category
    menu_items_data = parse_menu_items(menu_file)

    # Step 3: Create Workbook and 'HH Proposal' Sheet (No changes needed here)
    workbook = openpyxl.Workbook()
    worksheet = workbook.active
    worksheet.title = 'HH Proposal'

    # Step 4: Define styles (No changes needed here)
    red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
    light_yellow_fill = PatternFill(start_color="FFFFD4", end_color="FFFFD4", fill_type="solid")
    bright_yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
    white_font = Font(color="FFFFFF", bold=True, size=14)
    bold_font = Font(bold=True)
    center_aligned = Alignment(horizontal='center', vertical='center', wrap_text=True)
    left_aligned = Alignment(horizontal='left', vertical='center', wrap_text=True)
    border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))

    # --- Populate 'HH Proposal' Sheet (No changes needed in this block) ---
    # Header
    worksheet.merge_cells('A1:I1')
    header_cell = worksheet['A1']; header_cell.value = "Hungry Hub PARTY PACK PROPOSAL"
    header_cell.fill = red_fill; header_cell.font = white_font; header_cell.alignment = center_aligned
    worksheet.row_dimensions[1].height = 25
    # Restaurant Name
    worksheet['A2'] = "Restaurant Name:"; worksheet['A2'].font = bold_font
    worksheet.merge_cells('B2:I2'); worksheet['B2'] = "!!!--- Please Change Restaurant Name Here ---!!!"
    worksheet['B2'].alignment = center_aligned; worksheet['B2'].font = Font(bold=True, color="FF0000")
    # Package Names Row 3
    worksheet['A3'] = "Package Name:"; worksheet['A3'].font = bold_font
    pack_definitions = [
        {"name": "Pack A", "columns": "B:C", "bundle_index": 0},
        {"name": "Pack B", "columns": "D:E", "bundle_index": 1},
        {"name": "Pack C", "columns": "F:G", "bundle_index": 2},
        {"name": "Pack D", "columns": "H:I", "bundle_index": 3}
    ]
    for pack in pack_definitions:
        cell_range = f"{pack['columns'].split(':')[0]}3:{pack['columns'].split(':')[1]}3"
        worksheet.merge_cells(cell_range)
        cell = worksheet[cell_range.split(':')[0]]
        bundle_name = menu_bundles[pack['bundle_index']].get('bundle_name', pack['name'])
        cell.value = bundle_name; cell.alignment = center_aligned; cell.font = bold_font
    # HH Selling Price (NET) Row 4
    worksheet['A4'] = "HH Selling Price (NET)"; worksheet['A4'].font = bold_font
    for pack in pack_definitions:
        col_start, col_end = pack['columns'].split(':')
        cell_range = f"{col_start}4:{col_end}4"; worksheet.merge_cells(cell_range)

        # *** FIX 1 START ***
        price_value_raw = menu_bundles[pack['bundle_index']].get('suggested_bundle_price', 0) # Default to number 0
        price = 0.0 # Initialize price
        try:
            if isinstance(price_value_raw, str):
                # If it's a string, clean it
                price = float(price_value_raw.replace('$', '').replace(',', ''))
            elif isinstance(price_value_raw, (int, float)):
                # If it's already a number, use it directly (ensure float)
                price = float(price_value_raw)
            else:
                # Handle unexpected types if necessary
                logging.warning(f"Unexpected type for suggested_bundle_price: {type(price_value_raw)}. Using 0.")
                price = 0.0
        except ValueError:
            logging.warning(f"Could not convert suggested_bundle_price '{price_value_raw}' to float. Using 0.")
            price = 0.0
        # *** FIX 1 END ***

        cell = worksheet[col_start + '4']
        cell.value = price
        cell.number_format = '#,##0'; cell.alignment = center_aligned; cell.fill = light_yellow_fill

    # Max Diners Row 5 (No changes needed here)
    worksheet['A5'] = "Max Diners / Set"; worksheet['A5'].font = bold_font
    for pack in pack_definitions:
        col_start, col_end = pack['columns'].split(':')
        cell_range = f"{col_start}5:{col_end}5"; worksheet.merge_cells(cell_range)
        diners = menu_bundles[pack['bundle_index']].get('number_of_diners', 0)
        cell = worksheet[col_start + '5']
        cell.value = diners; cell.number_format = '0'; cell.alignment = center_aligned; cell.fill = light_yellow_fill

    # Remarks Row 6 (No changes needed here)
    worksheet['A6'] = "Remarks"; worksheet['A6'].font = bold_font
    for pack in pack_definitions:
        col_start, col_end = pack['columns'].split(':')
        cell_range = f"{col_start}6:{col_end}6"; worksheet.merge_cells(cell_range)
        worksheet[col_start + '6'] = "1 Water / Person"; worksheet[col_start + '6'].alignment = center_aligned

    # Menu Section Header Row 7 (No changes needed here)
    worksheet.merge_cells('A7:I7')
    menu_header = worksheet['A7']; menu_header.value = "Menu Section (portions from each section) - See Menu In Next Sheet"
    menu_header.fill = red_fill; menu_header.font = white_font; menu_header.alignment = center_aligned
    worksheet.row_dimensions[7].height = 25

    # Category Rows 8-11 - UPDATED with more detailed logging and improved handling
    group_rows = {"Category A": 8, "Category B": 9, "Category C": 10, "Category D": 11}
    for category_band, row in group_rows.items():
        worksheet[f'A{row}'] = category_band
        worksheet[f'A{row}'].font = bold_font
        
        # Add debug logging to see what's happening
        logging.info(f"Processing {category_band} on row {row}")
        
        for pack in pack_definitions:
            col_start, col_end = pack['columns'].split(':')
            cell_range = f"{col_start}{row}:{col_end}{row}"
            worksheet.merge_cells(cell_range)
            
            bundle_data = menu_bundles[pack['bundle_index']]
            bundle_name = bundle_data.get('bundle_name', f"Bundle {pack['bundle_index'] + 1}")
            
            # Debug what's in the bundle data
            logging.info(f"  Processing bundle: {bundle_name}")
            
            # Initialize portions to 0
            portions = 0
            
            # More robust checking for category_portions structure
            if 'category_portions' in bundle_data:
                if isinstance(bundle_data['category_portions'], dict):
                    # Debug what categories exist in the portions dict
                    logging.info(f"    Available categories: {list(bundle_data['category_portions'].keys())}")
                    
                    # Try exact match first
                    if category_band in bundle_data['category_portions']:
                        portions_raw = bundle_data['category_portions'][category_band]
                        # Ensure portions is an integer
                        try:
                            portions = int(portions_raw)
                        except (ValueError, TypeError):
                            portions = 0
                            logging.warning(f"    Could not convert '{portions_raw}' to int for {category_band}")
                    else:
                        logging.warning(f"    {category_band} not found in category_portions")
                else:
                    logging.warning(f"    category_portions is not a dict: {type(bundle_data['category_portions'])}")
            else:
                logging.warning(f"    No category_portions key found in bundle")
            
            # Set the cell value AFTER merging
            cell = worksheet[col_start + str(row)]
            cell.value = portions
            cell.number_format = '0'
            cell.alignment = center_aligned
            cell.fill = light_yellow_fill
            
            logging.info(f"    Set cell {col_start}{row} to {portions}")

    # Total Dishes Row 12 (No changes needed here)
    worksheet['A12'] = "Total Dishes"; worksheet['A12'].font = bold_font
    for pack in pack_definitions:
        col_start, col_end = pack['columns'].split(':')
        cell_range = f"{col_start}12:{col_end}12"; worksheet.merge_cells(cell_range)
        total_dishes = 0; bundle_data = menu_bundles[pack['bundle_index']]
        if 'category_portions' in bundle_data and isinstance(bundle_data['category_portions'], dict):
            total_dishes = sum(bundle_data['category_portions'].values())
        cell = worksheet[f"{col_start}12"]
        cell.value = total_dishes; cell.number_format = '0'; cell.alignment = center_aligned

    # Avg Price Header Row 13 (No changes needed here)
    worksheet.merge_cells('A13:I13')
    price_header = worksheet['A13']; price_header.value = "Average NET Selling Price / Discounts"
    price_header.fill = red_fill; price_header.font = white_font; price_header.alignment = center_aligned
    worksheet.row_dimensions[13].height = 25

    # Avg NET Selling Price Row 14 (Original Price)
    worksheet['A14'] = "Average NET Selling Price"; worksheet['A14'].font = bold_font
    for pack in pack_definitions:
        col_start, col_end = pack['columns'].split(':')
        cell_range = f"{col_start}14:{col_end}14"; worksheet.merge_cells(cell_range)

        # *** FIX 2 START ***
        price_value_raw = menu_bundles[pack['bundle_index']].get('original_bundle_price', 0) # Default to number 0
        price = 0.0 # Initialize price
        try:
            if isinstance(price_value_raw, str):
                # If it's a string, clean it
                price = float(price_value_raw.replace('$', '').replace(',', ''))
            elif isinstance(price_value_raw, (int, float)):
                # If it's already a number, use it directly (ensure float)
                price = float(price_value_raw)
            else:
                # Handle unexpected types if necessary
                logging.warning(f"Unexpected type for original_bundle_price: {type(price_value_raw)}. Using 0.")
                price = 0.0
        except ValueError:
            logging.warning(f"Could not convert original_bundle_price '{price_value_raw}' to float. Using 0.")
            price = 0.0
        # *** FIX 2 END ***

        cell = worksheet[col_start + '14']
        cell.value = price
        cell.number_format = '#,##0'; cell.alignment = center_aligned

    # Average Discount Row 15 (No changes needed here)
    worksheet['A15'] = "Average Discount"; worksheet['A15'].font = bold_font
    for pack in pack_definitions:
        col_start, col_end = pack['columns'].split(':')
        cell_range = f"{col_start}15:{col_end}{15}"; worksheet.merge_cells(cell_range)
        discount_value_raw = menu_bundles[pack['bundle_index']].get('discount_percentage', '0%')
        try:
            if isinstance(discount_value_raw, str):
                # If it's a string like "15%", clean and convert
                discount_val = float(discount_value_raw.replace('%', '')) / 100.0
                number_format = '0%'
            elif isinstance(discount_value_raw, (int, float)):
                # If it's already a number
                discount_val = float(discount_value_raw)
                # If the value is > 1, assume it's a percentage (e.g., 15 means 15%)
                if discount_val > 1:
                    discount_val = discount_val / 100.0
                number_format = '0%'
            else:
                # Handle unexpected type
                logging.warning(f"Unexpected type for discount_percentage: {type(discount_value_raw)}. Using 0%.")
                discount_val = 0.0
                number_format = '0%'
        except ValueError:
            # If conversion failed, use the original value as text
            discount_val = discount_value_raw
            number_format = '@'  # Text format
        cell = worksheet[col_start + '15']
        cell.value = discount_val; cell.number_format = number_format; cell.alignment = center_aligned

    # Net Price Per Person Row 16
    worksheet['A16'] = "Net Price / Person"; worksheet['A16'].font = bold_font
    for pack in pack_definitions:
        col_start, col_end = pack['columns'].split(':')
        cell_range = f"{col_start}16:{col_end}16"; worksheet.merge_cells(cell_range)

        # *** FIX 3 START (More robust handling) ***
        price_per_diner_val = menu_bundles[pack['bundle_index']].get('price_per_diner', '0') # Default to string '0'
        display_text = "0 / Person" # Default display text
        try:
            price_num = 0.0
            if isinstance(price_per_diner_val, str):
                # If it's a string like "$500" or "500", clean and convert
                price_num = float(price_per_diner_val.replace('$', '').replace(',', ''))
            elif isinstance(price_per_diner_val, (int, float)):
                # If it's already a number
                price_num = float(price_per_diner_val)
            else:
                # Handle unexpected type, keep price_num as 0.0
                logging.warning(f"Unexpected type for price_per_diner: {type(price_per_diner_val)}. Using 0.")

            display_text = f"{price_num:,.0f} / Person" # Format the extracted number

        except ValueError:
            # If conversion failed, use the original value (as string) in the display
            logging.warning(f"Could not convert price_per_diner '{price_per_diner_val}' to number. Displaying as is.")
            display_text = f"{price_per_diner_val} / Person"
        except Exception as e:
            # Catch potential formatting errors, though unlikely here
            logging.error(f"Error formatting price_per_diner '{price_per_diner_val}': {e}. Displaying as is.")
            display_text = f"{price_per_diner_val} / Person" # Fallback
        # *** FIX 3 END ***

        cell = worksheet[col_start + '16']
        cell.value = display_text
        cell.alignment = center_aligned; cell.fill = bright_yellow_fill; cell.font = bold_font
    # Adjustment Info Rows 17-18
    worksheet['A17'] = "You can Adjust"; worksheet['A17'].font = Font(italic=True)
    worksheet['A18'] = "Don't Adjust (Formula)"; worksheet['A18'].font = Font(italic=True)
    adjustable_rows = [4, 5]
    for row in adjustable_rows:
        for pack in pack_definitions:
            col_start, col_end = pack['columns'].split(':'); worksheet[f"{col_start}{row}"].fill = light_yellow_fill
    # Apply borders
    for row in worksheet.iter_rows(min_row=1, max_row=18, min_col=1, max_col=9):
        for cell in row:
            is_merged = any(cell.coordinate in merged_range for merged_range in worksheet.merged_cells.ranges)
            is_top_left = any(cell.coordinate == merged_range.coord.split(':')[0] for merged_range in worksheet.merged_cells.ranges if cell.coordinate in merged_range)
            if not is_merged or is_top_left: cell.border = border
    # Set column widths
    worksheet.column_dimensions['A'].width = 25
    for col_letter in ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']: worksheet.column_dimensions[col_letter].width = 15
    # --- End Populating 'HH Proposal' Sheet ---


    # *** Step 5: Create the menu sheet using the UPDATED sheet creator ***
    # This function now expects the new data structure and uses the semantic category
    create_menu_sheet(workbook, menu_items_data)

    # Step 6: Save the workbook (No changes needed here)
    try:
        output_abs_path = os.path.abspath(output_file)
        logging.info(f"Attempting to save workbook to: {output_abs_path}")
        workbook.save(output_abs_path)
        return f"Excel file '{output_file}' has been created successfully."
    except PermissionError:
        logging.error(f"Permission denied. Could not save '{output_file}'. Check if the file is open or permissions.")
        return f"Error: Permission denied saving '{output_file}'."
    except Exception as e:
        logging.error(f"Error saving Excel file '{output_file}': {e}", exc_info=True)
        return f"Error saving Excel file '{output_file}': {e}"


if __name__ == "__main__":
    bundle_json_path = './menu_bundles.json'
    # This is the text file generated by the *other* script
    menu_items_text_path = './categorized_menu_items.txt'
    output_excel_path = 'HH_Proposal_Generated.xlsx'

    # --- Create a dummy categorized_menu_items.txt with the NEW format if it doesn't exist ---
    if not os.path.exists(menu_items_text_path):
        logging.warning(f"'{menu_items_text_path}' not found. Creating a dummy file with the new format for testing.")
        try:
            with open(menu_items_text_path, 'w', encoding='utf-8') as f:
                f.write("===== MENU ITEMS BY PRICE CATEGORY (Generated by Script) =====\n")
                f.write("\n----- Category A ($600.00 - $750.00) -----\n")
                f.write("  $750.00 - Ribeye Steak (Main Course)\n")
                f.write("  $600.00 - Lobster Bisque (Appetizer)\n")
                f.write("\n----- Category B ($200.00 - $200.00) -----\n")
                f.write("  $200.00 - Grilled Chicken Sandwich (Main Course)\n")
                f.write("\n----- Category C ($150.00 - $150.00) -----\n")
                f.write("  $150.00 - Chocolate Lava Cake (Dessert)\n")
                f.write("  $150.00 - Caesar Salad (Appetizer)\n")
                f.write("\n----- Category D ($30.00 - $30.00) -----\n")
                f.write("  $30.00 - Side of Fries (Side Dish)\n")
                f.write("  $30.00 - Iced Tea (Beverage)\n")
            logging.info(f"Dummy '{menu_items_text_path}' created successfully.")
        except IOError as e:
            logging.error(f"Failed to create dummy menu items file: {e}")


    # Create a dummy menu_bundles.json if it doesn't exist (no changes needed here)
    if not os.path.exists(bundle_json_path):
        logging.warning(f"'{bundle_json_path}' not found. Creating a dummy file for testing.")
        # Dummy bundle data (same as before)
        dummy_bundles = [
            { "bundle_name": "Test Bundle A", "suggested_bundle_price": "$3100", "number_of_diners": 6, "category_portions": {"Category A": 3, "Category B": 4, "Category C": 2, "Category D": 1}, "original_bundle_price": "$3600", "discount_percentage": "14%", "price_per_diner": "$517" },
            { "bundle_name": "Test Bundle B", "suggested_bundle_price": "$4100", "number_of_diners": 8, "category_portions": {"Category A": 4, "Category B": 6, "Category C": 3, "Category D": 2}, "original_bundle_price": "$4900", "discount_percentage": "16%", "price_per_diner": "$513" },
            { "bundle_name": "Test Bundle C", "suggested_bundle_price": "$5100", "number_of_diners": 10, "category_portions": {"Category A": 5, "Category B": 8, "Category C": 4, "Category D": 3}, "original_bundle_price": "$6100", "discount_percentage": "16%", "price_per_diner": "$510" },
            { "bundle_name": "Test Bundle D", "suggested_bundle_price": "$6100", "number_of_diners": 12, "category_portions": {"Category A": 6, "Category B": 10, "Category C": 5, "Category D": 4}, "original_bundle_price": "$7300", "discount_percentage": "16%", "price_per_diner": "$508" }
        ]
        try:
            with open(bundle_json_path, 'w', encoding='utf-8') as f:
                json.dump(dummy_bundles, f, indent=4)
            logging.info(f"Dummy '{bundle_json_path}' created successfully.")
        except IOError as e:
            logging.error(f"Failed to create dummy bundles file: {e}")


    # Run the function to create the Excel file
    result = create_hungry_hub_proposal(
        json_file=bundle_json_path,
        menu_file=menu_items_text_path,
        output_file=output_excel_path
    )
    print(result) # Print final success/error message

2025-04-25 22:34:53,056 - INFO - Attempting to read bundles from: /Users/Cwkf_89/Documents/GitHub/hungryhub_automation/menu_bundles.json
2025-04-25 22:34:53,057 - INFO - Successfully loaded 4 bundles from ./menu_bundles.json
2025-04-25 22:34:53,057 - INFO - Attempting to read menu items from: /Users/Cwkf_89/Documents/GitHub/hungryhub_automation/categorized_menu_items.txt
2025-04-25 22:34:53,058 - INFO - Line 4: Found Price Band Header 'Category A'
2025-04-25 22:34:53,059 - INFO - Line 28: Found Price Band Header 'Category B'
2025-04-25 22:34:53,059 - INFO - Line 54: Found Price Band Header 'Category C'
2025-04-25 22:34:53,060 - INFO - Line 84: Found Price Band Header 'Category D'
2025-04-25 22:34:53,060 - INFO - Successfully parsed 87 menu items from ./categorized_menu_items.txt
2025-04-25 22:34:53,061 - INFO - --- Parsed Menu Items Summary (by Price Band) ---
2025-04-25 22:34:53,061 - INFO -   Category A: 22 items
2025-04-25 22:34:53,061 - INFO -   Category B: 24 items
2025-04-25 22:3

Excel file 'HH_Proposal_Generated.xlsx' has been created successfully.
